<a href="https://colab.research.google.com/github/akshatshah91/Game-AI/blob/master/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [7]:
episodes = 5000
tests = 10

In [86]:
class DQN(nn.Module):
  def __init__(self, input_size, output_size):
    super(DQN, self).__init__()
    self.input_size = input_size
    self.output_size = output_size
    self.hidden_size = int((input_size+output_size)/2)
    self.fc1 = nn.Linear(input_size, self.hidden_size)
    self.fc2 = nn.Linear(self.hidden_size, output_size)
    self.alpha = 0.001
    self.gamma = 0.95
    self.epsilon = 0.1
    self.batchSize = 10
    self.memory = []
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x
  def getAction(self, q):
    action = 0
    for x in range(1,self.output_size):
      if q[x] > q[action]:
        action = x
    return x
  def act(self, state):
    if np.random.rand() < self.epsilon:
      return random.randrange(self.output_size)
    else:
      return self.getAction(self.forward(state))
  def store(self, state, action, reward, nextState, gameOver):
    self.memory.append((state, action, reward, nextState, gameOver))
  def update(self):
    if len(self.memory) < self.batchSize:
      return
    minibatch = random.sample(self.memory, self.batchSize)
    stateActionValues = []
    expectedValues = []
    for s,a,r,nS,g in minibatch:
      stateActionValues.append(a)
      expectedValues.append(r + self.gamma*self.getAction(self.forward(torch.from_numpy(nS).type("torch.FloatTensor"))))
    loss = Variable(F.smooth_l1_loss(torch.Tensor(stateActionValues), torch.Tensor(expectedValues)), requires_grad=True)
    self.optimizer.zero_grad()
    loss.backward()
    for param in self.parameters():
      param.grad.data.clamp_(-1,1)
    self.optimizer.step()

In [92]:
def update(agent):
  if len(agent.memory) < agent.batchSize:
    return
  optimizer = optim.RMSprop(agent.parameters())
  minibatch = random.sample(agent.memory, agent.batchSize)
  stateActionValues = []
  expectedValues = []
  for s,a,r,nS,g in minibatch:
    stateActionValues.append(a)
    expectedValues.append(r + agent.gamma*agent.getAction(agent.forward(torch.from_numpy(nS).type("torch.FloatTensor"))))
    loss = Variable(F.smooth_l1_loss(torch.Tensor(stateActionValues), torch.Tensor(expectedValues)), requires_grad=True)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [94]:
env = gym.make("CartPole-v0").env
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQN(state_size, action_size)
for x in range(1, episodes+1):
  state = env.reset()
  gameOver = False
  totalReward = 0
  while not gameOver:
    action = agent.act(torch.from_numpy(state).type("torch.FloatTensor"))
    nextState,reward,gameOver,_ = env.step(action)
    agent.store(state, action, reward, nextState, gameOver)
    state = nextState
    totalReward += reward
    update(agent)
  if x%1000 is 0:
    print("Episode ", x, ": ", totalReward, sep='')

<generator object Module.parameters at 0x7f9e85f79360>


KeyboardInterrupt: ignored